## Compare detected face locations in different pliers face detection methods


### Tools that detect faces:

* Google Cloud Vision API
* Clarifai
* pliers itself


### Common measures
* boundaries of faces


### Ways to assess similarity
* Eucledian distance between coordinates?

In [1]:
import numpy as np
from os.path import join as opj
from pliers.extractors import (ClarifaiAPIImageExtractor,
                               FaceRecognitionFaceLocationsExtractor, 
                               GoogleVisionAPIFaceExtractor,
                               merge_results)
from pliers.stimuli import ImageStim
from pliers.filters import FrameSamplingFilter

from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from matplotlib import patches as patches

from moviepy.editor import *

W0808 14:46:03.666767 140736167261056 warnings.py:110] /Users/Mai/anaconda/envs/neuroscout/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])

W0808 14:46:03.668051 140736167261056 warnings.py:110] /Users/Mai/anaconda/envs/neuroscout/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])

W0808 14:46:03.668951 140736167261056 warnings.py:110] /Users/Mai/anaconda/envs/neuroscout/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [2]:
def plot_boundingBox(img, rect_coords, savename='', title=''):
    """
    Plots face bounding boxes on image.
    
    params
    ------
    
    returns
    -------
    fix,ax : figure and axis for resulting plot
    
    """
    # Plot image
    fig,ax = plt.subplots(1)
    if isinstance(img, str):
        img = mpimg.imread(img)
    imgplot = ax.imshow(img)

    # For each api, add bounding boxes
#     allowed_api = ['builtin', 'clarifai', 'google']
#     if api not in allowed_api:
#         raise ValueError(f'expected api specification from on in {allowed_api}, however I got "{api}".') 
   
    
    for c in rect_coords:
        rect = patches.Rectangle((c[0], c[1]), c[2], c[3],
                             linewidth=2,
                             edgecolor='r',
                             facecolor='none',
                            )
        ax.add_patch(rect)

    # turn off axis    
    plt.axis('off')
    
    plt.title(title)
    
    # save
    if not savename:
        plt.show()
    else:
        plt.savefig(savename)
    
    return fig,ax

In [3]:
def extract_bounding(results,
                    api='builtin',
                    stim = 'image', 
                    x=None,
                    y=None):
    """
    Extract bounding box coordinates from a face extraction with pliers build-in tool
    
    params
    ------
    results: pandas dataframe, result of a .to_df() operation on an extraction result
    api: one of 'builtin', 'clarifai', 'google'
    stim: one of 'image' or 'video'
    x, y: stimulus dimensions in pixel
    
    returns
    -------
    coords: dictionary, with one key per face and coordinates in pixel. Order of coords:
            left, bottom, width, height (how matplotlib plots it)
    >>> extract_bounding(result_clarifai, api='clarifai', stimulus='image' x=444, y=600)
    """
    
    def get_wh(top, right, bottom, left):
        box_width = right - left
        box_height = top - bottom
        return box_width, box_height
    
    allowed_api = ['builtin', 'clarifai', 'google']
    if api not in allowed_api:
        raise ValueError(f'expected api specification from on in {allowed_api}, however I got "{api}".') 
    
    # initialize an exmpty dict
    coords = {}
    if api == 'builtin':
        assert x, y != None
        coords['faces'] = []
        for idx, i in results.iterrows():
            top, right, bottom, left = i['face_locations']
            box_width, box_height = get_wh(top, right, bottom, left)
            coords['faces'].append([left, bottom, box_width, box_height])
    
    # Let's deal with Clarifai
    elif api == 'clarifai':
        if stim == 'image':
            assert x, y != None
            coords['faces'] = []
            for idx, i in results.iterrows():
                    top, right, bottom, left = i['top_row'] * y, i['right_col'] * x, i['bottom_row'] * y, i['left_col'] * x
                    box_width, box_height = get_wh(top, right, bottom, left)
                    # extract coordinates and scale them to pixels
                    coords['faces'].append([left, bottom, box_width, box_height])
                
                
        elif stim == 'video':
            # return a dictionary of dictionaries, and index appropriately
            for idx, i in results.iterrows():
                f = i['stim_name']
                if f not in coords.keys():
                    # initialize a list to append to
                    coords[f] = []
                    top, right, bottom, left = i['ClarifaiAPIImageExtractor#top_row'] * y, \
                                                    i['ClarifaiAPIImageExtractor#right_col'] * x, \
                                                    i['ClarifaiAPIImageExtractor#bottom_row'] * y, \
                                                    i['ClarifaiAPIImageExtractor#left_col'] * x
                    box_width, box_height = get_wh(top, right, bottom, left)
                    coords[f].append([left, bottom, box_width, box_height])
                else:
                    # add to existing frame layer
                    coords[f].append([left, bottom, box_width, box_height])
                    
    # Let's deal with Google            
    elif api == 'google':
        if stim == 'image':
            coords['faces'] = []
            for idx, i in results.iterrows():
                co = [i['fdBoundingPoly_vertex4_x'], # bottom_left_x
                      i['fdBoundingPoly_vertex1_y'], # top_left_y
                      i['fdBoundingPoly_vertex3_x'] - \
                           i['fdBoundingPoly_vertex4_x'], # width: bottom_right_x - bottom_left_x
                      i['fdBoundingPoly_vertex3_y'] - \
                           i['fdBoundingPoly_vertex2_y'] # height: bottom_right_y - top_right_y
                      ]
                coords['faces'].append(co)
        elif stim == 'video':
            # return a dictionary of dictionaries, and index appropriately
            for idx, i in results.iterrows():
                f = i['stim_name']
                if f not in coords.keys():
                    # initialize a list to append to
                    coords[f] = []
                    coords[f].append([i['GoogleVisionAPIFaceExtractor#fdBoundingPoly_vertex4_x'], # bottom_left_x
                              i['GoogleVisionAPIFaceExtractor#fdBoundingPoly_vertex1_y'], # top_left_y
                              i['GoogleVisionAPIFaceExtractor#fdBoundingPoly_vertex3_x'] - \
                                   i['GoogleVisionAPIFaceExtractor#fdBoundingPoly_vertex4_x'], # width: bottom_right_x - bottom_left_x
                              i['GoogleVisionAPIFaceExtractor#fdBoundingPoly_vertex3_y'] - \
                                   i['GoogleVisionAPIFaceExtractor#fdBoundingPoly_vertex2_y'] # height: bottom_right_y - top_right_y
                          ])
                else:
                    # add to existing frame layer
                    coords[f].append([i['GoogleVisionAPIFaceExtractor#fdBoundingPoly_vertex4_x'], # bottom_left_x
                               i['GoogleVisionAPIFaceExtractor#fdBoundingPoly_vertex1_y'], # top_left_y
                               i['GoogleVisionAPIFaceExtractor#fdBoundingPoly_vertex3_x'] - \
                                   i['GoogleVisionAPIFaceExtractor#fdBoundingPoly_vertex4_x'], # width: bottom_right_x - bottom_left_x
                               i['GoogleVisionAPIFaceExtractor#fdBoundingPoly_vertex3_y'] - \
                                   i['GoogleVisionAPIFaceExtractor#fdBoundingPoly_vertex2_y'] # height: bottom_right_y - top_right_y
                              ])
            
    return coords

In [4]:
def write_annotatedVideo(img_list, savename, audio=None):
    """
    Creates video from images with specified duration.
    
    params
    ------
    img_list : list of list, with each inner list specifying a path to an image and a duration
    savename : str, where to save movie file
    audio : str, optional argument for adding audio to movie
    
    >>> img_list = [['img1.jpj', .5], ['img2.jpg', .5]]
    >>> extract_bounding(img_list, savename='ex_video.mp4', audio='ex_aud.mp3')
    """
    
    # generate clips
    clips_list = []
    for img in img_list:
        clip = ImageClip(img[0], duration=img[1])
        clips_list.append(clip)

    # concatenate image clips
    print('Making video...')

    # Concat clips
    newMovie = concatenate_videoclips(clips_list) 

    # Add audio
    if audio:
        audio_clip = AudioFileClip(audio)
        newMovie = newMovie.set_audio(audio_clip)

    # Write movie
    newMovie.write_videofile(savename, fps=24)


In [12]:
# define static test images (single and many faces)
img_pth = opj('../', 'data', 'obama.jpg')
img_pth_many = opj('../', 'data', 'thai_people.jpg')
stim = ImageStim(img_pth)
stim_many = ImageStim(img_pth_many)


In [ ]:
# the results of the face detection are given relative to stimulus size. Let's get the image dimensions in pixel
y, x = stim.data.shape[:2]
print(f'the one-face picture is {x} pixel x {y} pixel in size')

y2, x2 = stim_many.data.shape[:2]
print(f'the many-face picture is {x2} pixel x {y2} pixel in size')

In [ ]:
# quick overview of the pictures
plt_img = mpimg.imread(img_pth)
plt_img2 = mpimg.imread(img_pth_many)
plt.figure(1)
plt.subplot(211)
plt.imshow(plt_img)

plt.subplot(212)
plt.imshow(plt_img2)

plt.axis('off')
plt.show()


### pliers face detection

In [ ]:
ext_pliers = FaceRecognitionFaceLocationsExtractor()
# for single face
result_pliers = ext_pliers.transform(stim).to_df()
# for many faces stimulus
result_pliers_many = ext_pliers.transform(stim_many).to_df()

In [ ]:
# extract faces for single and multi-face images from pliers-builtin, and plot them
for res, im, x_dim, y_dim in [(result_pliers, img_pth, x, y), (result_pliers_many, img_pth_many, x2, y2)]:
    d = extract_bounding(res, x=x_dim, y=y_dim)
    for k, i in d.items():
       # top, right, bottom, left = i
        box_width = right-left
        box_height = top-bottom
      #  coords = [[left, bottom, box_width, box_height]]
        plot_boundingBox(im, i, title='Pliers builtin')

In [ ]:
i

### clarifai face detection

In [ ]:
# the clarifai extraction needs a model and an api key
model='face' 
ext_clarifai = ClarifaiAPIImageExtractor(api_key='d53d5755b7514b87877df990f2d0bbd4',
                                         model=model)
result_clarifai = ext_clarifai.transform(stim).to_df()

# for many faces
result_clarifai_many = ext_clarifai.transform(stim_many).to_df()

In [ ]:
# extract faces for single and multi-face images from pliers-builtin, and plot them
for res, im, x_dim, y_dim in [(result_clarifai, img_pth, x, y), (result_clarifai_many, img_pth_many, x2, y2)]:
    d = extract_bounding(res, api='clarifai', x=x_dim, y=y_dim)
    for k, i in d.items():
        top, right, bottom, left = i
        box_width = right-left
        box_height = top-bottom
        coords = [[left, bottom, box_width, box_height]]
        plot_boundingBox(im, coords, 'Clarifai')

### Google Cloud vision API face detection

In [ ]:
#ext_google = GoogleVisionAPIFaceExtractor(discovery_file='/home/adina/NeuroHackademy-02c15db15c2a.json')
ext_google = GoogleVisionAPIFaceExtractor(discovery_file='/Users/Mai/NeuroHackademy-02c15db15c2a.json')
#result_google = ext_google.transform(stim_many).to_df()
results_google = ext_google.transform(stim).to_df()
results_google

In [ ]:
# Google has "wide" and "narrow" bounding boxes. Here we get the wide bounding box

#result_google.to_dict(orient='records')
# vertex coordinates are in the same scale as the original image.
# vertices are in order top-left, top-right, bottom-right, bottom-left.
top_left_x = results_google['boundingPoly_vertex1_x'][0]
top_right_x = results_google['boundingPoly_vertex2_x'][0]
bottom_right_x = results_google['boundingPoly_vertex3_x'][0]
bottom_left_x = results_google['boundingPoly_vertex4_x'][0]

top_left_y = results_google['boundingPoly_vertex1_y'][0]
top_right_y = results_google['boundingPoly_vertex2_y'][0]
bottom_right_y = results_google['boundingPoly_vertex3_y'][0]
bottom_left_y = results_google['boundingPoly_vertex4_y'][0]

print(top_left_x, top_right_x, bottom_right_x, bottom_left_x)
print(top_left_y, top_right_y, bottom_right_y, bottom_left_y)

In [ ]:
# # Plot bounding on image 
box_width = bottom_right_x - bottom_left_x
box_height =  bottom_right_y - top_right_y
coords_google_wide = [[bottom_left_x, top_left_y, box_width, box_height]]

plot_boundingBox(img_pth, coords_google_wide, title='Google: wide face bounding box')

In [ ]:
# Google has "wide" and "narrow" bounding boxes. Here we get the narrow bounding box

# vertex coordinates are in the same scale as the original image.
# vertices are in order top-left, top-right, bottom-right, bottom-left.
top_left_x = results_google['fdBoundingPoly_vertex1_x'][0]
top_right_x = results_google['fdBoundingPoly_vertex2_x'][0]
bottom_right_x = results_google['fdBoundingPoly_vertex3_x'][0]
bottom_left_x = results_google['fdBoundingPoly_vertex4_x'][0]

top_left_y = results_google['fdBoundingPoly_vertex1_y'][0]
top_right_y = results_google['fdBoundingPoly_vertex2_y'][0]
bottom_right_y = results_google['fdBoundingPoly_vertex3_y'][0]
bottom_left_y = results_google['fdBoundingPoly_vertex4_y'][0]

print(top_left_x, top_right_x, bottom_right_x, bottom_left_x)
print(top_left_y, top_right_y, bottom_right_y, bottom_left_y)

In [ ]:

# Plot bounding on image 
box_width = bottom_right_x - bottom_left_x
box_height =  bottom_right_y - top_right_y
coords_google_narrow = [[bottom_left_x, top_left_y, box_width, box_height]]
plot_boundingBox(img_pth, coords_google_narrow, title = 'Google: narrow face bounding box')


### Compare different face bounding boxes

In [ ]:
# let's start with looking at the coords
print('pliers: ' + str(coords_pliers))
print('clarifai: ' + str(coords_clarifai))
print('google (wide): ' + str(coords_google_wide))
print('google (narrow): ' + str(coords_google_narrow))

In [ ]:
# Plot on the same figure

# Make a dictionary with coords
face_apis = ['pliers', 'clarifai', 'google_wide', 'google_narrow']
coord_dict = dict(zip(face_apis, [coords_pliers, coords_clarifai, coords_google_wide, coords_google_narrow]))


### Detect faces in a video

In [8]:
# Output dir
out_dir = opj('../', 'output')

# Path to video
video_pth = opj('../', 'data', 'obama_speech.mp4')

# Sample 2 frames per second
sampler = FrameSamplingFilter(hertz=2)
frames = sampler.transform(video_pth)


In [20]:
# Video face extraction with Google
api = 'google'

# Extract using google API
ext_google = GoogleVisionAPIFaceExtractor(discovery_file='/Users/Mai/NeuroHackademy-02c15db15c2a.json')
results_google = ext_google.transform(frames)
results_google = merge_results(results_google, )

# get bounding boxes for every frame
coords_dict = extract_bounding(results_google, api=api, stim='video')

for i in range(frames.n_frames):
    # get this frame
    f = frames.get_frame(i)
    f_data = f.data
    f_name = f.name

    # get coords
    coords = coords_dict[f_name]
    
    # plot img with box and save    
    savename = opj(out_dir, 'img_' + str(i).zfill(3) + '_' + api + '.jpg')
    #plot_boundingBox(f_data, coords, savename)

W0808 15:07:07.721459 140736167261056 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/Users/Mai/anaconda/envs/neuroscout/lib/python3.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Mai/anaconda/envs/neuroscout/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Mai/anaconda/envs/neuroscout/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in

In [27]:
coords_all = {'google': coords_dict}

In [26]:
coords_dict

{'frame[0]': [[58, 54, 50, 50], [174, 33, 53, 53]],
 'frame[6]': [[165, 23, 52, 52], [59, 54, 50, 50]],
 'frame[12]': [[174, 24, 52, 52], [73, 50, 50, 50]],
 'frame[18]': [[184, 24, 52, 52], [67, 49, 50, 50]],
 'frame[24]': [[182, 25, 53, 52], [57, 52, 50, 50]],
 'frame[30]': [[171, 26, 56, 56], [56, 52, 50, 50]],
 'frame[36]': [[166, 27, 54, 55], [58, 52, 50, 50]],
 'frame[42]': [[170, 32, 55, 55], [59, 52, 50, 50]],
 'frame[48]': [[174, 31, 54, 54], [60, 52, 50, 50]],
 'frame[54]': [[175, 26, 53, 53], [60, 54, 50, 50]],
 'frame[60]': [[178, 24, 53, 53], [64, 52, 50, 50]],
 'frame[66]': [[66, 53, 50, 50], [179, 25, 53, 53]],
 'frame[72]': [[64, 53, 50, 50], [182, 25, 53, 53]],
 'frame[78]': [[180, 24, 52, 52], [63, 53, 50, 50]],
 'frame[84]': [[61, 55, 50, 50], [177, 27, 54, 54]],
 'frame[90]': [[174, 30, 55, 56], [60, 55, 50, 50]],
 'frame[96]': [[174, 29, 55, 56], [60, 55, 50, 50]],
 'frame[102]': [[165, 26, 54, 54], [61, 55, 50, 50]]}

In [6]:
# Video face extraction with Clarifai
api = 'clarifai'

model='face' 
ext_clarifai = ClarifaiAPIImageExtractor(api_key='d53d5755b7514b87877df990f2d0bbd4',
                                         model=model)
result_clarifai = ext_clarifai.transform(frames)
result_clarifai = merge_results(result_clarifai)

coords_dict = extract_bounding(results_clarifai, api=api, stim='video')

# for i in range(frames.n_frames):
#     # get this frame
#     f = frames.get_frame(i)
#     f_data = f.data
#     f_name = f.name

#     # get coords
#     coords = coords_dict[f_name]
    
#     # plot img with box and save    
#     savename = opj(out_dir, 'img_' + str(i).zfill(3) + '_' + api + '.jpg')
#     plot_boundingBox(f_data, coords, savename)

1it [00:06,  6.92s/it]
Stim: 18it [00:00, 94254.02it/s]


,filename,source_file,order,history,object_id,stim_name,duration,class,onset,ClarifaiAPIImageExtractor#bottom_row,ClarifaiAPIImageExtractor#left_col,ClarifaiAPIImageExtractor#right_col,ClarifaiAPIImageExtractor#top_row
0,NaN,/Users/Mai/Projects/tutorials/pliers/natstim_n...,NaN,VideoStim->FrameSamplingFilter/VideoFrameColle...,0,frame[0],0.50,VideoFrameStim,0.0,0.377024,0.551481,0.689992,0.112567
1,NaN,/Users/Mai/Projects/tutorials/pliers/natstim_n...,NaN,VideoStim->FrameSamplingFilter/VideoFrameColle...,1,frame[0],0.50,VideoFrameStim,0.0,0.413301,0.209890,0.319592,0.207180
2,NaN,/Users/Mai/Projects/tutorials/pliers/natstim_n...,NaN,VideoStim->FrameSamplingFilter/VideoFrameColle...,0,frame[6],0.50,VideoFrameStim,0.5,0.326510,0.529731,0.667629,0.083615
3,NaN,/Users/Mai/Projects/tutorials/pliers/natstim_n...,NaN,VideoStim->FrameSamplingFilter/VideoFrameColle...,1,frame[6],0.50,VideoFrameStim,0.5,0.417082,0.209207,0.319483,0.199690
4,NaN,/Users/Mai/Projects/tutorials/pliers/natstim_n...,NaN,VideoStim->FrameSamplingFilter/VideoFrameColle...,0,frame[12],0.50,VideoFrameStim,1.0,0.330235,0.559489,0.694852,0.086252
5,NaN,/Users/Mai/Projects/tutorials/pliers/natstim_n...,NaN,VideoStim->FrameSamplingFilter/VideoFrameColle...,1,frame[12],0.50,VideoFrameStim,1.0,0.404252,0.253766,0.368218,0.193471
6,NaN,/Users/Mai/Projects/tutorials/pliers/natstim_n...,NaN,VideoStim->FrameSamplingFilter/VideoFrameColle...,0,frame[18],0.50,VideoFrameStim,1.5,0.325365,0.591074,0.728219,0.085286
7,NaN,/Users/Mai/Projects/tutorials/pliers/natstim_n...,NaN,VideoStim->FrameSamplingFilter/VideoFrameColle...,1,frame[18],0.50,VideoFrameStim,1.5,0.401676,0.237146,0.352100,0.190108
8,NaN,/Users/Mai/Projects/tutorials/pliers/natstim_n...,NaN,VideoStim->FrameSamplingFilter/VideoFrameColle...,0,frame[24],0.50,VideoFrameStim,2.0,0.339815,0.585553,0.727233,0.084134
9,NaN,/Users/Mai/Projects/tutorials/pliers/natstim_n...,NaN,VideoStim->FrameSamplingFilter/VideoFrameColle...,1,frame[24],0.50,VideoFrameStim,2.0,0.410235,0.199929,0.313857,0.201012


In [ ]:
# Generate movies

# get list of clips and durations
img_dir = opj('../', 'output')
img_list = []
for i in range(18):
    img_list.append([opj(img_dir,'img_'+str(i).zfill(3)+'.jpg'), .5 ])

savename = opj(out_dir, 'obama_speech_annotated.mp4')
audio = '../data/obama_speech_audio.mp3'
write_annotatedVideo(img_list, savename=savename, audio=audio)

In [ ]:
b = None
if b:
    print('a')